### Moises Quiroz Diaz

## Desafío - Clasificación desde la econometría



* Para poder realizar esta actividad debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el .zip

### Descripción


En esta sesión trabajaremos con:
* sbp : Presión Sanguínea Sistólica.
* tobacco : Promedio tabaco consumido por día.
* ldl : Lipoproteína de baja densidad.
* adiposity : Adiposidad.
* famhist : Antecedentes familiares de enfermedades cardiácas. (Binaria)
* types : Personalidad tipo A
* obesity : Obesidad.
* alcohol : Consumo actual de alcohol.
* age : edad.
* chd : Enfermedad coronaria. (dummy)


### Desafío 1: Preparar el ambiente de trabajo

* Importe las librerías básicas para el análisis de datos.
* Incluya los import base de statsmodels .
* Importe la base de datos southafricanheart.csv  y elimine la columna 'Unnamed: 0'

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import  mean_squared_error , r2_score 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
df = pd.read_csv('southafricanheart.csv')
df = df.drop("Unnamed: 0", axis=1)
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


### Desafío 2
A continuación se presenta el siguiente modelo a estimar:

$$\log\left(\frac{p({\bf chd=1})}{1 - p({\bf chd=1})}\right)={{\beta }_{0} + {\beta }_{1} \cdot {famhist}}$$


Para ello ejecute los siguientes pasos:

1. Recodifique famhist a dummy, asignando 1 a la categoría minoritaria.
2. Utilice smf.logit para estimar el modelo.
3. Implemente una función inverse_logit que realize el mapeo de log-odds a probabilidad.
4. Con el modelo estimado, responda lo siguiente:
    * ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?
    * ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
    * ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?

Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados.
tip: Utilice ${\beta }/4$

grafique ambas curvas con lmplot



In [2]:
df.famhist.value_counts()

Absent     270
Present    192
Name: famhist, dtype: int64

In [3]:
tmp = pd.get_dummies(df,columns=['famhist'])
tmp = tmp.drop('famhist_Absent', axis =1)
tmp.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_Present
0,160,12.00,5.73,23.11,49,25.30,97.20,52,1,1
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1,0
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0,1
3,170,7.50,6.41,38.03,51,31.99,24.26,58,1,1
4,134,13.60,3.50,27.78,60,25.99,57.34,49,1,1


In [4]:
modelo = smf.logit('chd ~ famhist_Present',tmp).fit()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


In [5]:
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      460
Method:                           MLE   Df Model:                            1
Date:                Mon, 08 Jul 2019   Pseudo R-squ.:                 0.05740
Time:                        11:10:54   Log-Likelihood:                -280.95
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 4.937e-09
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1690      0.143     -8.169      0.000      -1.449      -0.889
famhist_Present     1.1690      0.203      5.751      0.000       0.771       1.567
===================================================================================
"""

In [6]:
def inverse_logit(x):
    return 1/(1+np.exp(-x))

In [7]:
def probabilidad(x,modelo):
    yhat = modelo.params['Intercept'] + modelo.params['famhist_Present']*x
    return inverse_logit(yhat)

¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?

In [8]:
probabilidad(1,modelo)

0.4999999999999999

¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?

In [9]:
print('a diferencia del modelo anterior, la probabilidad de un individuo sin antecedentes familiares es:')
probabilidad(0,modelo)

a diferencia del modelo anterior, la probabilidad de un individuo sin antecedentes familiares es:


0.237037037037037

¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?

In [10]:
print('Si se tienen antecedentes se obtiene que aumenta la probabilidad en:')
probabilidad(1,modelo)-probabilidad(0,modelo)

Si se tienen antecedentes se obtiene que aumenta la probabilidad en:


0.2629629629629629

In [11]:
tmp.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_Present
0,160,12.00,5.73,23.11,49,25.30,97.20,52,1,1
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1,0
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0,1
3,170,7.50,6.41,38.03,51,31.99,24.26,58,1,1
4,134,13.60,3.50,27.78,60,25.99,57.34,49,1,1


### Desafío 3

Implemente un modelo con la siguiente forma

$$\log\left(\frac{p({\bf chd=1})}{1 - p({\bf chd=1})}\right)={{\beta }_{0} + \sum_{j=0}^N {\beta }_{j} \cdot {X}}$$



1. Depure el modelo manteniendo las variables con significancia estadística al 95%.
2. Compare los estadísticos de bondad de ajuste entre ambos.
3. Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.


In [12]:
modelo2 = smf.logit('chd ~ famhist_Present + sbp + tobacco + ldl+adiposity+ typea+obesity+ alcohol + age ',tmp).fit()
modelo2.summary()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Mon, 08 Jul 2019   Pseudo R-squ.:                  0.2080
Time:                        11:10:55   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.055e-22
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.1507      1.308     -4.701      0.000      -8.715      -3.587
famhist_Present     0.9254      0.228      4.061      0.000       0.479       1.372
sbp                 0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco             0.0794      0.027      2.984      0.003       0.027       0.132
ldl                 0.1739      0.060      2.915      0.004       0.057       0.291
adiposity           0.0186      0.029      0.635      0.526      -0.039       0.076
typea               0.0396      0.012      3.214      0.001       0.015       0.064
obesity            -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol             0.0001      0.004      0.027      0.978      -0.009       0.009
age                 0.0452      0.012      3.728      0.000       0.021       0.069
===================================================================================
"""

In [13]:
modelo3 = smf.logit('chd ~ famhist_Present + tobacco + ldl+ typea + age ',tmp).fit()
modelo3.summary()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Mon, 08 Jul 2019   Pseudo R-squ.:                  0.2020
Time:                        11:10:55   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.554e-24
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.4464      0.921     -7.000      0.000      -8.251      -4.642
famhist_Present     0.9082      0.226      4.023      0.000       0.466       1.351
tobacco             0.0804      0.026      3.106      0.002       0.030       0.131
ldl                 0.1620      0.055      2.947      0.003       0.054       0.270
typea               0.0371      0.012      3.051      0.002       0.013       0.061
age                 0.0505      0.010      4.944      0.000       0.030       0.070
===================================================================================
"""

El segundo modelo tiene menor R-square, pero se observa que este baja poco independiente que se le quitaron varias variables, caso parecido con el parametro de verosimilitud, finalmente se concluye que los modelos son muy parecidos en rendimiento

In [14]:
def probabilidad2(x,modelo):
    yhat = modelo.params['Intercept'] + modelo.params['famhist_Present']*x+modelo.params['age']*tmp.age.mean()+modelo.params['typea']*tmp.typea.mean()+modelo.params['tobacco']*tmp.tobacco.mean()+modelo.params['ldl']*tmp.ldl.mean()
    return inverse_logit(yhat)
print('la nueva probabilidad con antecedentes es:',probabilidad2(1,modelo3),"y antes:",probabilidad(1,modelo))
print('la nueva probabilidad sin antecedentes es:',probabilidad2(0,modelo3),"y antes:",probabilidad(0,modelo))

la nueva probabilidad con antecedentes es: 0.4141865908182175 y antes: 0.4999999999999999
la nueva probabilidad sin antecedentes es: 0.22185998096946388 y antes: 0.237037037037037


### Desafío 4: Estimación de perfiles
A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas
a probabilidades con inverse_logit . Los perfiles a estimar son los siguientes:
* La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
* La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.
* La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

* La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.

In [15]:
yhat = modelo3.params['Intercept'] + modelo3.params['famhist_Present']*tmp.famhist_Present.mean()+modelo3.params['age']*tmp.age.mean()+modelo3.params['typea']*tmp.typea.mean()+modelo3.params['tobacco']*tmp.tobacco.mean()+modelo3.params['ldl']*tmp.ldl.mean()
print('la probabilidad es:',inverse_logit(yhat))



la probabilidad es: 0.29370927481586945


* La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

In [16]:
yhat = modelo3.params['Intercept'] + modelo3.params['famhist_Present']*tmp.famhist_Present.mean()+modelo3.params['age']*tmp.age.mean()+modelo3.params['typea']*tmp.typea.mean()+modelo3.params['tobacco']*tmp.tobacco.mean()+modelo3.params['ldl']*tmp.ldl.max()
print('la probabilidad es:',inverse_logit(yhat))



la probabilidad es: 0.6980443104466211


* La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

In [17]:
yhat = modelo3.params['Intercept'] + modelo3.params['famhist_Present']*tmp.famhist_Present.mean()+modelo3.params['age']*tmp.age.mean()+modelo3.params['typea']*tmp.typea.mean()+modelo3.params['tobacco']*tmp.tobacco.mean()+modelo3.params['ldl']*tmp.ldl.min()
print('la probabilidad es:',inverse_logit(yhat))



la probabilidad es: 0.18443595575404634
